In [1]:
!pip install numpy pandas opencv-python torch torchvision ultralytics pydicom matplotlib

  Using cached matplotlib-3.10.1-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached pillow-11.1.0-cp313-cp313-manylinux_2_28_x86_64.whl.metadata (9.1 kB)
  Using cached contourpy-1.3.1-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.57.0-cp313-cp313-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (102 kB)
  Using cached kiwisolver-1.4.8-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.2 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 3.8 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.6/766.6 MB 3.6 MB/s eta 0:00:00m eta 0:00:010:00:06
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00m eta 0:00:010:00:03
   ━━━━━━━━━━

In [2]:
import pandas as pd
import json

In [18]:
# Load the Excel file
xls_file = "/home/yukta/devel/wip/AIML/ultrasound-analysis/data/annotations/ObjectDetection.xlsx"
df = pd.read_excel(xls_file)
print(df.head())  # Show the first few rows to verify the structure


     fname   structure  h_min  w_min  h_max  w_max
0  168.png     thalami    178    171    244    261
1  168.png  nasal bone     96    308    111    349
2  168.png      palate    133    300    205    408
3  168.png  nasal skin     86    324     95    349
4  168.png   nasal tip     79    345     89    376


In [19]:
import json
import os

In [26]:
# Check if required columns exist
required_columns = {"fname", "structure", "h_min", "w_min", "h_max", "w_max"}
if not required_columns.issubset(df.columns):
    raise KeyError(f"Missing columns in Excel file: {required_columns - set(df.columns)}")

In [20]:
# COCO format initialization
coco_data = {
    "images": [],
    "annotations": [],
    "categories": []
}

In [27]:
# Define category mapping
category_mapping = {structure: idx for idx, structure in enumerate(df["structure"].unique(), 1)}
for structure, idx in category_mapping.items():
    coco_data["categories"].append({"id": idx, "name": structure})

In [21]:
category_map = {}
category_id = 1
image_id_map = {}
image_id = 1
annotation_id = 1

In [29]:

      # Process each row
image_id_map = {}
annotation_id = 1
for _, row in df.iterrows():
    img_filename = row["fname"]
    if img_filename not in image_id_map:
        image_id = len(image_id_map) + 1
        image_id_map[img_filename] = image_id
        coco_data["images"].append({"id": image_id, "file_name": img_filename})
    
    coco_data["annotations"].append({
        "id": annotation_id,
        "image_id": image_id_map[img_filename],
        "category_id": category_mapping[row["structure"]],
        "bbox": [row["w_min"], row["h_min"], row["w_max"] - row["w_min"], row["h_max"] - row["h_min"]],
        "iscrowd": 0
    })
    annotation_id += 1


In [30]:
# Save to JSON
coco_annotations_path = "annotations_coco.json"
with open(coco_annotations_path, "w") as f:
    json.dump(coco_data, f)

print(f"COCO annotations saved to {coco_annotations_path}!")

COCO annotations saved to annotations_coco.json!


In [12]:
import cv2
import os
import glob
from sklearn.model_selection import train_test_split


In [13]:
# Load image paths (Update path to reflect actual dataset location)
image_dir = "/home/yukta/devel/wip/AIML/ultrasound-analysis/data/images/DatasetForFetus"
image_paths = glob.glob(os.path.join(image_dir, "**", "*.png"), recursive=True)  # Recursive search for all PNG images

# Split dataset into training and testing sets
train_imgs, test_imgs = train_test_split(image_paths, test_size=0.2, random_state=42)

print(f"Training images: {len(train_imgs)}, Testing images: {len(test_imgs)}")


Training images: 1347, Testing images: 337


In [32]:
from ultralytics import YOLO

In [41]:
yolo_model = YOLO("yolov8n.pt")  # Load pre-trained YOLOv8 nano model

In [42]:
# Train the model
yolo_model.train(data="dataset.yaml", epochs=50, imgsz=640)

Ultralytics 8.3.101 🚀 Python-3.13.2 torch-2.6.0+cu124 CPU (AMD Ryzen 7 5700U with Radeon Graphics)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_b

train: Scanning /home/yukta/devel/wip/AIML/ultrasound-analysis/data/labels/Datas

train: WARNING ⚠️ No labels found in /home/yukta/devel/wip/AIML/ultrasound-analysis/data/labels/DatasetForFetus/Set1-Training&Validation Sets CNN/Standard.cache. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
train: WARNING ⚠️ Cache directory /home/yukta/devel/wip/AIML/ultrasound-analysis/data/labels/DatasetForFetus/Set1-Training&Validation Sets CNN is not writeable, cache not saved.
WARNING ⚠️ No labels found in /home/yukta/devel/wip/AIML/ultrasound-analysis/data/labels/DatasetForFetus/Set1-Training&Validation Sets CNN/Standard.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.



val: Scanning /home/yukta/devel/wip/AIML/ultrasound-analysis/data/labels/Dataset

val: WARNING ⚠️ No labels found in /home/yukta/devel/wip/AIML/ultrasound-analysis/data/labels/DatasetForFetus/Internal Test Set/Non-standard.cache. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
val: WARNING ⚠️ Cache directory /home/yukta/devel/wip/AIML/ultrasound-analysis/data/labels/DatasetForFetus/Internal Test Set is not writeable, cache not saved.
WARNING ⚠️ No labels found in /home/yukta/devel/wip/AIML/ultrasound-analysis/data/labels/DatasetForFetus/Internal Test Set/Non-standard.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
Plotting labels to runs/detect/train5/labels.jpg... 
zero-size array to reduction operation maximum which has no identity
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000769, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weig

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       6/50         0G          0      57.01          0          0        640: 1
                 Class     Images  Instances      Box(P          R      mAP50  m

                   all        156          0          0          0          0          0
WARNING ⚠️ no labels found in detect set, can not compute metrics without labels

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size



/home/yukta/devel/wip/AIML/jupyter_env/lib/python3.13/site-packages/ultralytics/utils/metrics.py:654: RuntimeWarning: Mean of empty slice.
  i = smooth(f1_curve.mean(0), 0.1).argmax()  # max F1 index
/home/yukta/devel/wip/AIML/jupyter_env/lib/python3.13/site-packages/numpy/_core/_methods.py:139: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
       7/50         0G          0      48.17          0          0        640: 1
                 Class     Images  Instances      Box(P          R      mAP50  m


RuntimeError: torch.cat(): expected a non-empty list of Tensors

In [ ]:
# Ensure annotations exist
if not os.path.exists(coco_annotations_path):
    raise FileNotFoundError(f"Annotations file {coco_annotations_path} does not exist!")


In [ ]:
# Train model on ultrasound data
yolo train data=dataset.yaml model=yolov8n.pt epochs=50 imgsz=640


In [ ]:
# Perform inference
sample_image = "sample_ultrasound.png"  # Ensure this image exists
if not os.path.exists(sample_image):
    raise FileNotFoundError(f"Sample image {sample_image} not found!")

results = yolo_model(sample_image)

# Visualize results
results.show()

In [ ]:
coco_data = {"images": [], "annotations": [], "categories": []}


In [ ]:
with open("annotations_coco.json", "w") as f:
    json.dump(coco_data, f)

print("COCO annotations saved!")

In [ ]:
# Perform inference
sample_image = "/home/yukta/devel/wip/AIML/ultrasound-analysis/data/images/sample_ultrasound.png"  # Update path if needed
if not os.path.exists(sample_image):
    raise FileNotFoundError(f"Sample image {sample_image} not found!")

results = yolo_model(sample_image)

# Visualize results
results.show()